In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches
import psycopg2
import datetime
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *



start='2020-01-01'
end='2020-01-11'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_union_data(test_date):
    print 'test_data:' , test_date

    sql_text = """
    
    
        -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );
    """
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
             {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }
        ]
    }
    
    run(spark, ingest_msg, sql_text)

    est_stored_data_count = spark.sql('''
SELECT COUNT(*) from (
SELECT DISTINCT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id FROM (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id, category_id, col_free_app_download, col_paid_app_download, col_revenue 
FROM 
(
SELECT * , 
Coalesce(free_app_download, 0) AS col_free_app_download,
Coalesce(paid_app_download, 0) AS col_paid_app_download,
Coalesce(revenue, 0) AS col_revenue
FROM unified_rank_filter_data_free_app_download 
) as t_1
WHERE  
(col_free_app_download > 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW')


OR (col_free_app_download > 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')
) as t_2

) as t_3
''').take(1)

    total_count = spark.sql('''
SELECT count(*) from (select distinct country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id from unified_rank_filter_data_free_app_download ) as prod

''').take(1)

    print est_stored_data_count[0][0], total_count[0][0]
    print "diff is :" , float(est_stored_data_count[0][0]) / float(total_count[0][0])

    

    
sc.parallelize(map(test_union_data, dates), 1)

# spark.sql("select * from unified_rank_filter_data_free_app_download").show()


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches
import psycopg2
import datetime
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *



start='2020-01-01'
end='2020-01-11'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_union_data(test_date):
    print 'test_data:' , test_date

    sql_text = """
    
    
        -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );
    
    

    
    """
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
             {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }
        ]
    }
    
    run(spark, ingest_msg, sql_text)

    est_stored_data_count = spark.sql('''
SELECT COUNT(*) from (
SELECT DISTINCT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id FROM (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id, category_id, col_free_app_download, col_paid_app_download, col_revenue 
FROM 
(
SELECT * , 
Coalesce(free_app_download, 0) AS col_free_app_download,
Coalesce(paid_app_download, 0) AS col_paid_app_download,
Coalesce(revenue, 0) AS col_revenue
FROM unified_rank_filter_data_free_app_download 
) as t_1
WHERE  
((col_free_app_download > 1000 or col_paid_app_download > 1000) and col_revenue <= 1000 and country_code != 'WW') 
OR ((col_free_app_download > 1000 or col_revenue > 1000) and col_paid_app_download <= 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and (col_paid_app_download > 1000 or col_revenue > 1000) and country_code != 'WW')


OR ((col_free_app_download > 4000 or col_paid_app_download > 4000) and col_revenue <= 4000 and country_code == 'WW')
OR ((col_free_app_download > 4000 or col_revenue > 4000) and col_paid_app_download <= 4000   and country_code == 'WW')
OR (col_free_app_download <= 4000 and (col_paid_app_download > 4000 or col_revenue > 4000) and country_code == 'WW')
) as t_2

) as t_3
''').take(1)

    total_count = spark.sql('''
SELECT count(*) from (select distinct country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, app_id from unified_rank_filter_data_free_app_download ) as prod

''').take(1)

    print est_stored_data_count[0][0], total_count[0][0]
    print "diff is :" , float(est_stored_data_count[0][0]) / float(total_count[0][0])

    

    
sc.parallelize(map(test_union_data, dates), 1)

# spark.sql("select * from unified_rank_filter_data_free_app_download").show()


In [0]:

spark.sql('''

SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue 
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from unified_rank_filter_data_free_app_download 
) as t_1
WHERE 
(col_free_app_download > 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue > 1000 and country_code != 'WW')

OR (col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW')



OR (col_free_app_download > 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')

''').show()



In [0]:

spark.sql('''
select country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, free_app_download, paid_app_download, revenue from unified_rank_filter_data_free_app_download
except all
select country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, free_app_download, paid_app_download, revenue from (
select country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, free_app_download, paid_app_download, revenue  from(
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue , free_app_download, paid_app_download, revenue
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from unified_rank_filter_data_free_app_download 
) as t_1
WHERE 
(col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
)
as t 

union all

select country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, free_app_download, paid_app_download, revenue from (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue, free_app_download, paid_app_download, revenue
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from unified_rank_filter_data_free_app_download 
) as t_1
WHERE 
(col_free_app_download > 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue > 1000 and country_code != 'WW')

OR (col_free_app_download > 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue > 4000 and country_code == 'WW')

OR (col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW')

OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')

) as t_2 

) as t_3



''').show()



In [0]:

spark.sql("select * from unified_rank_filter_data_free_app_download where  country_code='US' and device_code='ios-phone' order by paid_app_download desc ").show()

In [0]:

spark.sql('''
select *  from  (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue 
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from  unified_rank_filter_data_free_app_download
) as t_1
WHERE 
(col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
) as t

''').show()



In [0]:

spark.sql('''
select count(*)  from  (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue 
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from  unified_rank_filter_data_free_app_download
) as t_1
WHERE 
(col_free_app_download > 1000 and col_paid_app_download > 1000 and col_revenue >1000 and country_code != 'WW') 
OR (col_free_app_download > 4000 and col_paid_app_download > 4000 and col_revenue > 4000 and country_code == 'WW')
) as t

''').show()



In [0]:

spark.sql('''
select * from (
SELECT country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, category_id, app_id, col_free_app_download, col_paid_app_download, col_revenue 
from 
(
select * , 
coalesce(free_app_download, 0) AS col_free_app_download,
coalesce(paid_app_download, 0) AS col_paid_app_download,
coalesce(revenue, 0) AS col_revenue
from unified_rank_filter_data_free_app_download 
) as t_1
WHERE 
(col_free_app_download > 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW') 
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue > 1000 and country_code != 'WW')

OR (col_free_app_download > 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue > 4000 and country_code == 'WW')

OR (col_free_app_download <= 1000 and col_paid_app_download <= 1000 and col_revenue > 1000 and country_code != 'WW')
OR (col_free_app_download > 1000 and col_paid_app_download <= 1000 and col_revenue <= 1000 and country_code != 'WW')
OR (col_free_app_download <= 1000 and col_paid_app_download > 1000 and col_revenue <= 1000 and country_code != 'WW')

OR (col_free_app_download <= 4000 and col_paid_app_download <= 4000 and col_revenue > 4000 and country_code == 'WW')
OR (col_free_app_download > 4000 and col_paid_app_download <= 4000 and col_revenue <= 4000 and country_code == 'WW')
OR (col_free_app_download <= 4000 and col_paid_app_download > 4000 and col_revenue <= 4000 and country_code == 'WW')

) as t_2 ''')

In [0]:

|          US|  ios-phone|                 null|                   11|         23|           null|               null|     100000| 924691770|             null|             1000|  10889|       null|           null|      daily|2020-01-01|


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.10.254  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select * from store.store_est_fact_v6 where date in ('2020-01-01', '2020-01-02', '2020-01-03','2020-01-04','2020-01-05') and country_code='US' and device_code='ios-phone' and app_id =924691770 
EOF



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select * from plproxy.execute_select_nestloop(\$proxy\$ 
    select sum(estimate) as cnt 
    from ( select distinct app_id,feed_id,store_id,estimate,device_id from aa.app_store_daily_estimate_1
    where date in ('2020-01-01', '2020-01-02', '2020-01-03','2020-01-04','2020-01-05')   and  feed_id in (1) and store_id=143441 and app_id=924691770 ) as prod
\$proxy\$) tbl (cnt bigint);



EOF

In [0]:
%%sh
